<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [446]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt

In [338]:
truck_rides = pd.read_csv(r'C:\Users\ovnanyan_gm\Desktop\Проекты\Грузовики\truck_rides_2022_04_06_with_stop_type_v2.csv',
                         index_col=0)
                             
truck_rides.head(1)

,tt,plate,vehicleId,geometry,ride_num,reg_seq,transit,distance_km,typeTS,massa_dop,АЗС,Склад,Парковка,stop_class
0,2022-04-06 00:23:12+03:00,X158CB178,215015.0,POINT (37.732704 55.592934),1,"Москва, Область",False,46.0,NaN,18000,NaN,NaN,NaN,Н/Д


In [339]:
truck_rides = truck_rides[['plate', 'vehicleId', 'geometry', 'ride_num', 'reg_seq']].copy()

In [318]:
truck_rides_1_2 = truck_rides.query('ride_num in (1, 2)')

In [340]:
truck_rides[truck_rides['plate'] == 'A020CP799']

,plate,vehicleId,geometry,ride_num,reg_seq
180203,A020CP799,128481.0,POINT (37.75778 55.540714),1,Область
180204,A020CP799,128481.0,POINT (37.770172 55.537422),2,"Область, Москва"
180205,A020CP799,128481.0,POINT (37.4705 55.816383),3,"Москва, Область"
180206,A020CP799,128481.0,POINT (37.757915 55.54038),4,Область
180207,A020CP799,128481.0,POINT (37.757835 55.540306),5,Область
180208,A020CP799,128481.0,POINT (37.757835 55.540306),6,Область
180209,A020CP799,128481.0,POINT (37.757854 55.540318),7,Область
180210,A020CP799,128481.0,POINT (37.757843 55.540306),8,Область


In [168]:
from shapely import wkt
truck_rides_1_2['g'] = truck_rides_1_2.geometry.apply(wkt.loads)
truck_rides_1_2 = gpd.GeoDataFrame(truck_rides_1_2, geometry=truck_rides_1_2['g'])

C:\Users\OVNANY~1\AppData\Local\Temp/ipykernel_21648/3379764798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truck_rides_1_2['g'] = truck_rides_1_2.geometry.apply(wkt.loads)


In [41]:
%time gdf.to_file(r'C:\Users\ovnanyan_gm\Python\Грузовой тр-т\data\1_2.shp', encoding='utf-8')

Wall time: 18.9 s


In [169]:
truck_rides_1_2.head()

,plate,vehicleId,geometry,ride_num,reg_seq,g
0,X158CB178,215015.0,POINT (37.73270 55.59293),1,"Москва, Область",POINT (37.732704 55.592934)
1,X158CB178,215015.0,POINT (37.76411 55.86253),2,Москва,POINT (37.764107 55.86253)
3,A895BO799,77900.0,POINT (37.43115 55.67592),1,Москва,POINT (37.43115 55.67592)
4,A895BO799,77900.0,POINT (37.43076 55.67603),2,Москва,POINT (37.430756 55.676025)
7,X955XA777,132872.0,POINT (37.99860 55.69935),1,Область,POINT (37.9986 55.699345)


In [170]:
truck_rides_from_to = truck_rides_1_2.assign(idx=truck_rides_1_2.groupby('ride_num').cumcount()).pivot_table(
    index=['idx', 'plate', 'vehicleId'], columns='ride_num', values=['g', 'reg_seq'], aggfunc='sum'
).rename(columns={1:'from', 2:'to'})
truck_rides_from_to.head()

g  \
ride_num                                        from   
idx plate     vehicleId                                
0   X158CB178 215015.0   POINT (37.732704 55.592934)   
1   A895BO799 77900.0      POINT (37.43115 55.67592)   
2   X955XA777 132872.0     POINT (37.9986 55.699345)   
3   E079ET799 202433.0   POINT (37.270264 55.979637)   
4   E666KM48  186217.0   POINT (37.557503 55.844246)   

                                                              reg_seq           
ride_num                                          to             from       to  
idx plate     vehicleId                                                         
0   X158CB178 215015.0    POINT (37.764107 55.86253)  Москва, Область   Москва  
1   A895BO799 77900.0    POINT (37.430756 55.676025)           Москва   Москва  
2   X955XA777 132872.0   POINT (37.998905 55.699184)          Область  Область  
3   E079ET799 202433.0   POINT (37.270264 55.979637)          Область  Область  
4   E666KM48  186217.0     POINT (37.55741 55.84414)           Москва   Москва

In [171]:
truck_rides_from_to.columns = truck_rides_from_to.columns.droplevel()
truck_rides_from_to.head()

,,ride_num,from,to,from,to
idx,plate,vehicleId,,,,
0,X158CB178,215015.0,POINT (37.732704 55.592934),POINT (37.764107 55.86253),"Москва, Область",Москва
1,A895BO799,77900.0,POINT (37.43115 55.67592),POINT (37.430756 55.676025),Москва,Москва
2,X955XA777,132872.0,POINT (37.9986 55.699345),POINT (37.998905 55.699184),Область,Область
3,E079ET799,202433.0,POINT (37.270264 55.979637),POINT (37.270264 55.979637),Область,Область
4,E666KM48,186217.0,POINT (37.557503 55.844246),POINT (37.55741 55.84414),Москва,Москва


In [172]:
truck_rides_from_to = truck_rides_from_to.reset_index()

In [174]:
truck_rides_from_to.head()

ride_num,idx,plate,vehicleId,from,to,from,to
0,0,X158CB178,215015.0,POINT (37.732704 55.592934),POINT (37.764107 55.86253),"Москва, Область",Москва
1,1,A895BO799,77900.0,POINT (37.43115 55.67592),POINT (37.430756 55.676025),Москва,Москва
2,2,X955XA777,132872.0,POINT (37.9986 55.699345),POINT (37.998905 55.699184),Область,Область
3,3,E079ET799,202433.0,POINT (37.270264 55.979637),POINT (37.270264 55.979637),Область,Область
4,4,E666KM48,186217.0,POINT (37.557503 55.844246),POINT (37.55741 55.84414),Москва,Москва


In [ ]:
truck_rides_1_2 = gpd.GeoDataFrame(truck_rides_1_2, geometry=truck_rides_1_2['g'])

In [173]:
%time truck_rides_from_to.to_file(r'C:\Users\ovnanyan_gm\Python\Грузовой тр-т\data\1_2.shp', encoding='utf-8')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [411]:
polygon = gpd.read_file(r'C:\Users\ovnanyan_gm\Desktop\Проекты\Грузовики\from_to_id.shp', encoding='UTF-8')
polygon = gpd.GeoDataFrame(polygon, geometry=polygon['geometry'])
polygon = polygon.to_crs("EPSG:3857")
polygon.head()

,tt,plate,vehicleId,ride_num,reg_seq,transit,distance_k,typeTS,massa_dop,АЗС,Склад,Парко,stop_class,id,left,top,right,bottom,geometry
0,2022-04-06 00:23:12+03:00,X158CB178,215015.0,1,"Москва, Область",False,46.0,None,18000,None,None,None,Н/Д,5231.0,419852.041204,6.161904e+06,420352.041204,6.161404e+06,MULTIPOINT (4200385.396 7477803.458)
1,2022-04-06 21:05:57+03:00,X158CB178,215015.0,2,Москва,False,42.1,None,18000,None,None,None,Н/Д,5631.0,422352.041204,6.191904e+06,422852.041204,6.191404e+06,MULTIPOINT (4203881.162 7531097.799)
2,2022-04-06 10:25:37+03:00,A895BO799,77900.0,1,Москва,False,1.7,Грузовой тягач седельный,18000,None,Склад,None,Склад,1716.0,400852.041204,6.171404e+06,401352.041204,6.170904e+06,MULTIPOINT (4166816.558 7494169.155)
3,2022-04-06 12:32:30+03:00,A895BO799,77900.0,2,Москва,False,0.6,Грузовой тягач седельный,18000,None,Склад,None,Склад,1716.0,400852.041204,6.171404e+06,401352.041204,6.170904e+06,MULTIPOINT (4166772.698 7494189.884)
4,2022-04-06 04:50:13+03:00,E666KM48,186217.0,1,Москва,False,0.1,грузовой бортовой,3500,None,None,None,Н/Д,3243.0,409352.041204,6.189904e+06,409852.041204,6.189404e+06,MULTIPOINT (4180882.109 7527471.716)


In [412]:
truck_rides = polygon[['plate', 'vehicleId', 'id', 'geometry', 'ride_num', 'reg_seq']].copy()
truck_rides_1_2 = truck_rides.query('ride_num in (1, 2)')
truck_rides_1_2.head()

,plate,vehicleId,id,geometry,ride_num,reg_seq
0,X158CB178,215015.0,5231.0,MULTIPOINT (4200385.396 7477803.458),1,"Москва, Область"
1,X158CB178,215015.0,5631.0,MULTIPOINT (4203881.162 7531097.799),2,Москва
2,A895BO799,77900.0,1716.0,MULTIPOINT (4166816.558 7494169.155),1,Москва
3,A895BO799,77900.0,1716.0,MULTIPOINT (4166772.698 7494189.884),2,Москва
4,E666KM48,186217.0,3243.0,MULTIPOINT (4180882.109 7527471.716),1,Москва


In [413]:
truck_rides_from_to = truck_rides_1_2[truck_rides_1_2['plate'].duplicated(keep=False)].sort_values(
    by=['plate', 'ride_num'])
truck_rides_from_to.head(6)

,plate,vehicleId,id,geometry,ride_num,reg_seq
53309,2401PA77,101516.0,3693.0,MULTIPOINT (4185376.189 7536219.344),1,Москва
53310,2401PA77,101516.0,3693.0,MULTIPOINT (4185376.189 7536219.344),2,Москва
12393,A002CE777,258109.0,3037.0,MULTIPOINT (4178463.248 7546647.576),1,Область
12394,A002CE777,258109.0,3037.0,MULTIPOINT (4178478.388 7546679.379),2,Область
5926,A002CY190,251725.0,6872.0,MULTIPOINT (4216016.767 7491311.052),1,Область
5927,A002CY190,251725.0,6872.0,MULTIPOINT (4216015.988 7491310.657),2,Область


In [414]:
truck_rides_from_to = truck_rides_from_to.drop_duplicates(subset=['plate', 'ride_num'])

In [415]:
# truck_rides_from_to = truck_rides_from_to.assign(idx=truck_rides_from_to.groupby('ride_num').cumcount())
truck_rides_from_to['id'] = truck_rides_from_to['id'].astype('str')
truck_rides_from_to.head(6)

,plate,vehicleId,id,geometry,ride_num,reg_seq
53309,2401PA77,101516.0,3693.0,MULTIPOINT (4185376.189 7536219.344),1,Москва
53310,2401PA77,101516.0,3693.0,MULTIPOINT (4185376.189 7536219.344),2,Москва
12393,A002CE777,258109.0,3037.0,MULTIPOINT (4178463.248 7546647.576),1,Область
12394,A002CE777,258109.0,3037.0,MULTIPOINT (4178478.388 7546679.379),2,Область
5926,A002CY190,251725.0,6872.0,MULTIPOINT (4216016.767 7491311.052),1,Область
5927,A002CY190,251725.0,6872.0,MULTIPOINT (4216015.988 7491310.657),2,Область


In [416]:
truck_rides_from_to[truck_rides_from_to['plate'] == 'H815XT77']

,plate,vehicleId,id,geometry,ride_num,reg_seq


In [417]:
truck_rides_from_to = truck_rides_from_to.pivot_table(
    index=['plate', 'vehicleId'], columns='ride_num', values=['id'], aggfunc='sum'
).rename(columns={1:'from', 2:'to'})
truck_rides_from_to

id        
ride_num               from      to
plate     vehicleId                
2401PA77  101516.0   3693.0  3693.0
A002CE777 258109.0   3037.0  3037.0
A002CY190 251725.0   6872.0  6872.0
A003HA777 96085.0    3266.0  3266.0
A003KM67  242710.0   1249.0  1249.0
...                     ...     ...
Y996KY777 129065.0   1130.0  1130.0
Y996TY69  228119.0   1819.0  1819.0
Y998AM799 84557.0    4245.0  4245.0
Y998TM197 129444.0   5735.0  2581.0
Y999OX77  71644.0    2097.0  1051.0

[22226 rows x 2 columns]

In [418]:
truck_rides_from_to.columns = truck_rides_from_to.columns.droplevel()
truck_rides_from_to = truck_rides_from_to.reset_index()
truck_rides_from_to

ride_num,plate,vehicleId,from,to
0,2401PA77,101516.0,3693.0,3693.0
1,A002CE777,258109.0,3037.0,3037.0
2,A002CY190,251725.0,6872.0,6872.0
3,A003HA777,96085.0,3266.0,3266.0
4,A003KM67,242710.0,1249.0,1249.0
...,...,...,...,...
22221,Y996KY777,129065.0,1130.0,1130.0
22222,Y996TY69,228119.0,1819.0,1819.0
22223,Y998AM799,84557.0,4245.0,4245.0
22224,Y998TM197,129444.0,5735.0,2581.0


In [438]:
truck_rides_count = truck_rides_from_to.groupby(['from', 'to']).agg(
    {'plate':'nunique'})
truck_rides_count = truck_rides_count.query("plate>2").sort_values(by='plate', ascending=False).reset_index()
truck_rides_count

,from,to,plate
0,2799.0,2799.0,99
1,5217.0,5217.0,89
2,5825.0,5825.0,61
3,2508.0,2508.0,60
4,6380.0,6380.0,55
...,...,...,...
1343,6325.0,6325.0,3
1344,4120.0,4119.0,3
1345,6324.0,6323.0,3
1346,4124.0,4125.0,3


In [440]:
truck_rides_count['tag'] = 0
for i in range(len(truck_rides_count)):
    if truck_rides_count.loc[i, 'from'] == truck_rides_count.loc[i, 'to']:
        truck_rides_count.loc[i, 'tag'] = 'stop'
    else: 
        truck_rides_count.loc[i, 'tag'] = 'ride'
truck_rides_count.head()

,from,to,plate,tag
0,2799.0,2799.0,99,stop
1,5217.0,5217.0,89,stop
2,5825.0,5825.0,61,stop
3,2508.0,2508.0,60,stop
4,6380.0,6380.0,55,stop


In [443]:
truck_rides_count.query("tag=='ride'").head(40)

,from,to,plate,tag
120,5571.0,5570.0,18,ride
257,5015.0,5107.0,11,ride
334,5570.0,5571.0,10,ride
364,5111.0,5019.0,9,ride
543,2508.0,1947.0,7,ride
548,5222.0,4115.0,6,ride
554,3100.0,3099.0,6,ride
567,990.0,897.0,6,ride
644,5616.0,5617.0,5,ride
647,4831.0,4924.0,5,ride


840